In [1]:
import os
import numpy as np
import tensorflow as tf
import trainer_2d
import trainer_3d
import data_utils
from PIL import Image

In [2]:
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import pandas as pd

In [3]:
import importlib
importlib.reload(trainer_2d)
importlib.reload(trainer_3d)
importlib.reload(data_utils)

<module 'data_utils' from '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/src/data_utils.py'>

In [4]:
DATA_PATH = '/Users/nastya/Documents/ucu/thesis/data/drive_data'

In [6]:
test_data = pd.read_csv('/Users/nastya/Documents/ucu/thesis/voxelmorph/drive_new/vm_gan_drive_new/test_split.csv')

In [7]:
pairs = []

for index, row in test_data.iterrows():
    target_image = row['targets'].replace('/content/drive_data', DATA_PATH)
    source_image = row['sources'].replace('/content/drive_data', DATA_PATH)
    pairs.append((target_image, source_image))

In [8]:
pairs[0:3]

[('/Users/nastya/Documents/ucu/thesis/data/drive_data/pairs_0/target_mask.jpg',
  '/Users/nastya/Documents/ucu/thesis/data/drive_data/pairs_0/source_masks/source_mask_15.jpg'),
 ('/Users/nastya/Documents/ucu/thesis/data/drive_data/pairs_7/target_mask.jpg',
  '/Users/nastya/Documents/ucu/thesis/data/drive_data/pairs_7/source_masks/source_mask_5.jpg'),
 ('/Users/nastya/Documents/ucu/thesis/data/drive_data/pairs_14/target_mask.jpg',
  '/Users/nastya/Documents/ucu/thesis/data/drive_data/pairs_14/source_masks/source_mask_9.jpg')]

In [9]:
source_images = [(data_utils.load_color_image(pair[1], (256, 256)) > 127).view(np.uint8) * 255 for pair in pairs]
target_images = [(data_utils.load_color_image(pair[0], (256, 256)) > 127).view(np.uint8) * 255 for pair in pairs]

In [11]:
metrics_data = pd.DataFrame(columns=['mse', 'dice'])

In [12]:
model_path = '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/drive_prediction/model'
general_prediction_path = '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/drive_prediction/prediction'

for idx, (source_image, target_image) in tqdm(enumerate(zip(source_images, target_images))):
    
    prediction_path = os.path.join(general_prediction_path, f'pair_{idx}')

    source_image = trainer_2d._load_data_from_array(source_image)
    target_image = trainer_2d._load_data_from_array(target_image)
    
    source_label = trainer_2d._one_hot_label(source_image, labels=(0, 255))
    target_label = trainer_2d._one_hot_label(target_image, labels=(0, 255))

    training_data = {'target_image': target_image, 'target_label': target_label,
                 'source_image': source_image, 'source_label': source_label,
                 'target_affine': np.eye(4), 'target_header': None
                 }

    
    with tf.Graph().as_default(), tf.device('/cpu:0'):
        trainer = trainer_2d.Trainer(input_size=(source_image.shape[1], source_image.shape[2]), num_channels=1, num_classes=2,
                          demons_type='diffeomorphic', demons_force='moving', normalization='z-score',
                          max_length=2., exp_steps=7)

        metrics, save_path = trainer.train(training_data, training_iters=100,
                                           display_steps=0,
                                           model_path=model_path,
                                           prediction_path=prediction_path)

        new_row = {'mse': trainer.final_mse, 'dice': trainer.final_dice}
        metrics_data.loc[len(metrics_data)] = new_row

0it [00:00, ?it/s]

Instructions for updating:
Use fn_output_signature instead


2024-04-25 15:30:57,566 From /opt/homebrew/lib/python3.9/site-packages/tensorflow/python/util/deprecation.py:660: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Use fn_output_signature instead
2024-04-25 15:30:58,542 Removing '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/drive_prediction/prediction/pair_0'
2024-04-25 15:30:58,547 Removing '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/drive_prediction/model'
2024-04-25 15:30:58,549 Allocating '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/drive_prediction/prediction/pair_0'
2024-04-25 15:30:58,550 Allocating '/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/drive_prediction/model'
2024-04-25 15:30:58.564049: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-04-25 15:30:58,658 Start optimization with demons configuration: demons force: moving

In [13]:
metrics_data

,mse,dice
0,0.030894,0.743174
1,0.041303,0.731219
2,0.034648,0.752099
3,0.033908,0.760268
4,0.039294,0.709840
...,...,...
75,0.037194,0.803805
76,0.036960,0.690742
77,0.031049,0.804441
78,0.029241,0.765532


In [15]:
metrics_data['mse'].mean()

0.036995888

In [16]:
metrics_data['dice'].mean()

0.7244423

In [14]:
metrics_data.to_csv('/Users/nastya/Documents/ucu/thesis/diffeomorphic_demons/drive_prediction/metrics.csv', index=False)